In [ ]:

"""
Estimate sensitivity coefficients for COPD & IHD deaths
w.r.t. PM2.5, O3, NO2, temperature, VMT, NDVI.

Model: log(1 + deaths) ~ pm25_mean + o3_mean + no2_mean + tavg + vmt + ndvi
"""

import pandas as pd
import numpy as np
import statsmodels.api as sm
import json

CSV_PATH = "Final_Master_with_preds.csv"   # adjust path as needed
df = pd.read_csv(CSV_PATH)

# Parse month column (expects something like 'YYYY-MM')
if "month" in df.columns:
    df["month"] = pd.to_datetime(df["month"])
else:
    raise ValueError("Expected a 'month' column in interpolated.csv")


cutoff = pd.Timestamp("2025-08-01")
df_hist = df[df["month"] < cutoff].copy()

print(f"Total rows in interpolated.csv: {len(df)}")
print(f"Rows used for historical fitting (< {cutoff.date()}): {len(df_hist)}")

feature_cols = [
    "pm25_mean",
    "o3_mean",
    "no2_mean",
    "tavg",
    "vmt",
    "ndvi"
]

target_cols = ["copd_deaths", "ihd_deaths"]

for col in feature_cols + target_cols:
    if col not in df_hist.columns:
        raise ValueError(f"Column '{col}' not found in interpolated.csv")

# Drop rows with missing values in any of the used columns
df_hist = df_hist.dropna(subset=feature_cols + target_cols)
print(f"Rows after dropping NaNs in features+targets: {len(df_hist)}")


X = df_hist[feature_cols].copy()

# Log-transform deaths only (log1p handles zeros safely)
df_hist["log_copd"] = np.log1p(df_hist["copd_deaths"])
df_hist["log_ihd"]  = np.log1p(df_hist["ihd_deaths"])

y_copd = df_hist["log_copd"]
y_ihd  = df_hist["log_ihd"]

print("\nNon-finite counts BEFORE cleaning in X:")
print(~np.isfinite(X).sum())

print("\nNon-finite counts in y_copd / y_ihd:")
print("y_copd:", (~np.isfinite(y_copd)).sum())
print("y_ihd :", (~np.isfinite(y_ihd)).sum())

# Drop any remaining non-finite rows just in case
mask_finite = np.isfinite(X).all(axis=1) & np.isfinite(y_copd) & np.isfinite(y_ihd)
X = X.loc[mask_finite]
y_copd = y_copd.loc[mask_finite]
y_ihd = y_ihd.loc[mask_finite]

print(f"\nRows after dropping any remaining non-finite values: {len(X)}")

# Add intercept
X_const = sm.add_constant(X)

# Final sanity checks
assert np.isfinite(X_const.values).all(), "Still have non-finite values in X_const!"
assert np.isfinite(y_copd.values).all(),  "Still have non-finite values in y_copd!"
assert np.isfinite(y_ihd.values).all(),   "Still have non-finite values in y_ihd!"

# 6. Fit OLS models
model_copd = sm.OLS(y_copd, X_const).fit()
model_ihd  = sm.OLS(y_ihd, X_const).fit()

print("\n=== COPD model (log deaths ~ raw pollutants/activity/weather) ===")
print(model_copd.summary())

print("\n=== IHD model (log deaths ~ raw pollutants/activity/weather) ===")
print(model_ihd.summary())

def coeff_or_zero(model, name):
    """Safe extraction of a coefficient; 0 if missing."""
    try:
        return float(model.params[name])
    except KeyError:
        return 0.0

sensitivity = {
    "COPD": {
        "pm25_mean": coeff_or_zero(model_copd, "pm25_mean"),
        "o3_mean":   coeff_or_zero(model_copd, "o3_mean"),
        "no2_mean":  coeff_or_zero(model_copd, "no2_mean"),
        "tavg":      coeff_or_zero(model_copd, "tavg"),
        "vmt":       coeff_or_zero(model_copd, "vmt"),
        "ndvi":      coeff_or_zero(model_copd, "ndvi"),
    },
    "IHD": {
        "pm25_mean": coeff_or_zero(model_ihd, "pm25_mean"),
        "o3_mean":   coeff_or_zero(model_ihd, "o3_mean"),
        "no2_mean":  coeff_or_zero(model_ihd, "no2_mean"),
        "tavg":      coeff_or_zero(model_ihd, "tavg"),
        "vmt":       coeff_or_zero(model_ihd, "vmt"),
        "ndvi":      coeff_or_zero(model_ihd, "ndvi"),
    }
}

print(json.dumps(sensitivity, indent=2))

OUT_PATH = "sensitivity_coeffs.json"
with open(OUT_PATH, "w") as f:
    json.dump(sensitivity, f, indent=2)
print(f"\nSaved coefficients to {OUT_PATH}")
